## Oleh:
- Putu Gde Kenzie Carlen Mataram (71230994)
<br>
- Tara Tirtanata (71231056)

In [55]:
%pip install backtesting yfinance pandas_ta plotly scikit-learn numpy pandas

Note: you may need to restart the kernel to use updated packages.


In [56]:
import yfinance as yf
import pandas as pd
import numpy as np
import pandas_ta as ta
from sklearn.ensemble import RandomForestClassifier
from backtesting import Backtest, Strategy
import warnings

# Konfigurasi Tampilan
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
warnings.filterwarnings('ignore')

In [57]:
# ================= CONFIGURATION =================
# 1. Daftar Aset untuk Diadu
TICKERS = ['QQQ', 'SPY', 'NVDA', 'TSLA', 'AAPL', 'AMD', 'AMZN', 'MSFT', 'GOOGL', 'META', 'GC=F'] 

# 2. Timeframe Eksekusi untuk Diuji
# M1 = Sangat agresif (Noise tinggi)
# M5 = Balance
# M15 = Lebih stabil (Sedikit sinyal tapi akurat)
TIMEFRAMES = ['1m', '5m', '15m']

# 3. Parameter Keamanan (Safety)
ADX_THRESHOLD = 25       # Minimum kekuatan trend (di bawah ini = DILARANG TRADING)
RISK_PER_TRADE = 0.02    # Risiko 2% per entry awal
LEVERAGE = 1.0           # Kita set 1.0 (Tanpa Leverage) dulu untuk melihat performa murni
CASH = 10000             # Modal Awal

print(f"Konfigurasi Siap: Menguji {len(TICKERS)} Aset pada {len(TIMEFRAMES)} Timeframe.")

Konfigurasi Siap: Menguji 11 Aset pada 3 Timeframe.


In [58]:
def get_data(symbol, interval):
    """
    Mengambil data sesuai batasan Yahoo Finance.
    - 1m: Max 7 hari
    - 5m, 15m: Max 60 hari
    """
    period = "7d" if interval == "1m" else "60d"
    
    try:
        df = yf.download(symbol, period=period, interval=interval, progress=False)
        if isinstance(df.columns, pd.MultiIndex): df.columns = df.columns.droplevel(1)
        df = df.rename(columns={'Adj Close': 'Close'})
        df = df[['Open', 'High', 'Low', 'Close', 'Volume']]
        df = df[df['Volume'] > 0]
        
        # Standarisasi Timezone ke UTC agar cocok saat mapping
        if df.index.tz is None: 
            df.index = df.index.tz_localize('UTC')
        else:
            df.index = df.index.tz_convert('UTC')
            
        return df
    except Exception as e:
        print(f"Gagal download {symbol} ({interval}): {e}")
        return None

def add_technical_features(df):
    df = df.copy()
    # Trend Filter (PENTING UNTUK KEAMANAN)
    adx = ta.adx(df['High'], df['Low'], df['Close'], length=14)
    df['ADX'] = adx[adx.columns[0]] # Mengambil kolom ADX utama
    
    # Trend Direction
    df['EMA_50'] = ta.ema(df['Close'], length=50)
    df['EMA_200'] = ta.ema(df['Close'], length=200)
    
    # Volatility untuk Stop Loss
    df['ATR'] = ta.atr(df['High'], df['Low'], df['Close'], length=14)
    
    # Momentum
    df['RSI'] = ta.rsi(df['Close'], length=14)
    
    # Bollinger Bands (Untuk ML features)
    bb = ta.bbands(df['Close'], length=20, std=2)
    df['BB_Width'] = bb[bb.columns[2]] # Bandwidth
    
    return df.dropna()

In [59]:
# CELL 4: Brain Engine (Machine Learning) - REVISED (ANTI-ERROR)

def train_macro_model(symbol):
    """
    Melatih model Random Forest pada data Harian/Jam 
    untuk menentukan bias jangka panjang (Bullish/Bearish).
    """
    print(f"   [ML] Downloading data training untuk {symbol}...")
    # Download data 1 tahun (max period)
    df = yf.download(symbol, period="1y", interval="1h", progress=False)
    
    # Cleaning MultiIndex column issues from yfinance
    if isinstance(df.columns, pd.MultiIndex): df.columns = df.columns.droplevel(1)
    
    # Pastikan nama kolom benar
    df = df.rename(columns={'Adj Close': 'Close'})
    
    # Feature Engineering Sederhana
    df['RSI'] = ta.rsi(df['Close'], length=14)
    df['Return'] = df['Close'].pct_change()
    
    # Hati-hati dengan Volume change, jika volume 0 bisa jadi inf
    # Kita tambahkan 1e-9 (angka sangat kecil) untuk menghindari pembagian dengan nol
    df['Vol_Change'] = (df['Volume'] + 1e-9).pct_change()
    
    # --- FIX ERROR: CLEANING INFINITY & NAN ---
    # Mengganti nilai inf/-inf menjadi NaN, lalu didrop
    df = df.replace([np.inf, -np.inf], np.nan)
    df.dropna(inplace=True)
    # ------------------------------------------
    
    # Validasi Data Kosong setelah cleaning
    if len(df) < 50:
        raise ValueError(f"Data tidak cukup setelah cleaning ({len(df)} rows)")

    # Target: Apakah harga Close besok lebih tinggi?
    df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)
    
    features = ['RSI', 'Return', 'Vol_Change']
    X = df[features]
    y = df['Target']
    
    # Double check cleaning (Safety Net terakhir)
    X = X.replace([np.inf, -np.inf], 0).fillna(0)
    
    # Training
    model = RandomForestClassifier(n_estimators=100, min_samples_split=10, random_state=42)
    model.fit(X, y)
    
    # Kembalikan DataFrame dengan sinyal prediksi
    df['Macro_Signal'] = model.predict(X)
    
    if df.index.tz is None: df.index = df.index.tz_localize('UTC')
    else: df.index = df.index.tz_convert('UTC')
        
    return df[['Macro_Signal']]

In [60]:
class SmartPyramid(Strategy):
    # Parameter Optimal (Bisa di-tweak)
    atr_sl_multiplier = 2.0   # Stop Loss jarak 2x ATR
    atr_tp_multiplier = 4.0   # Take Profit (Opsional, kita pakai trailing)
    max_layers = 4            # Maksimal 4 lapis posisi
    adx_filter = 20       # JANGAN TRADE jika ADX < 25 (Pasar Sideways)
    
    def init(self):
        self.highest_price = 0
        self.stop_price = 0
    
    def next(self):
        price = self.data.Close[-1]
        atr = self.data.ATR[-1]
        adx = self.data.ADX[-1]
        macro_signal = self.data.Macro_Signal[-1] # Sinyal dari H1
        ema_50 = self.data.EMA_50[-1]
        
        # --- 1. EXIT LOGIC (Trailing Stop) ---
        if self.position.is_long:
            # Update Trailing Stop jika harga naik
            if price > self.highest_price:
                self.highest_price = price
                # Stop loss naik mengikuti harga (Trailing)
                new_stop = self.highest_price - (atr * self.atr_sl_multiplier)
                self.stop_price = max(self.stop_price, new_stop)
            
            # Eksekusi Exit
            if price < self.stop_price:
                self.position.close()
            
            # Hard Exit jika tren berbalik dibawah EMA 50
            if price < ema_50:
                self.position.close()

        # --- 2. ENTRY LOGIC (Hanya jika Posisi < Max Layers) ---
        # SAFETY CHECK: 
        # 1. Market harus Trending (ADX > 25)
        # 2. Macro (H1) harus Bullish
        # 3. Harga di atas EMA 50 (Uptrend)
        
        is_trending = adx > self.adx_filter
        is_uptrend = price > ema_50
        is_macro_bullish = macro_signal == 1
        
        if is_trending and is_uptrend and is_macro_bullish:
            
            # Entry Pertama
            if not self.position:
                self.buy(size=0.30) # Masuk 30% Equity
                self.highest_price = price
                self.stop_price = price - (atr * self.atr_sl_multiplier)
            
            # Pyramiding (Tambah Muatan)
            elif len(self.trades) < self.max_layers:
                last_entry = self.trades[-1].entry_price
                
                # Hanya nambah jika posisi sebelumnya sudah profit (Scale In Winners)
                if price > last_entry + (1.0 * atr):
                    self.buy(size=0.20) # Tambah 20%

In [61]:
summary_data = []

print("===================================================================")
print("              MULAI PROSES KOMPARASI MULTI-ASET & TF")
print("===================================================================\n")

for ticker in TICKERS:
    # 1. Siapkan Otak (Macro Model)
    print(f"--- Melatih Model Macro untuk {ticker} ---")
    try:
        macro_df = train_macro_model(ticker)
    except Exception as e:
        print(f"Skip {ticker}: Gagal train model ({e})")
        continue

    for tf in TIMEFRAMES:
        print(f"Processing: {ticker} pada Timeframe {tf}...")
        
        # 2. Siapkan Data Eksekusi (Micro)
        micro_df = get_data(ticker, tf)
        if micro_df is None or len(micro_df) < 50:
            print(f"  -> Data {ticker} {tf} tidak cukup.")
            continue
            
        micro_df = add_technical_features(micro_df)
        
        # 3. Gabungkan Sinyal Macro ke Micro
        # Forward fill sinyal H1 ke data menit
        micro_df['Macro_Signal'] = macro_df['Macro_Signal'].reindex(micro_df.index, method='ffill')
        micro_df.dropna(inplace=True)
        
        # 4. Jalankan Backtest
        try:
            bt = Backtest(micro_df, SmartPyramid, cash=CASH, commission=0.0005, margin=1.0)
            stats = bt.run()
            
            # 5. CETAK LAPORAN DETAIL (Sesuai Request)
            print(f"\n>>> REPORT: {ticker} [{tf}] <<<")
            print(stats) # Ini akan mencetak blok teks panjang seperti contoh Anda
            print("-" * 50)
            
            # Simpan ringkasan untuk perbandingan akhir
            summary_data.append({
                'Ticker': ticker,
                'Timeframe': tf,
                'Return [%]': stats['Return [%]'],
                'Win Rate [%]': stats['Win Rate [%]'],
                'Sharpe Ratio': stats['Sharpe Ratio'],
                'Max Drawdown [%]': stats['Max. Drawdown [%]'],
                '# Trades': stats['# Trades'],
                'Profit Factor': stats['Profit Factor'],
                'Duration': stats['Duration']
            })
            
        except Exception as e:
            print(f"  -> Error Backtest {ticker} {tf}: {e}")

print("\n\nProses Selesai.")

              MULAI PROSES KOMPARASI MULTI-ASET & TF

--- Melatih Model Macro untuk QQQ ---
   [ML] Downloading data training untuk QQQ...
Processing: QQQ pada Timeframe 1m...


Backtest.run:   0%|          | 0/2482 [00:00<?, ?bar/s]


>>> REPORT: QQQ [1m] <<<
Start                     2025-12-09 17:52...
End                       2025-12-17 20:59...
Duration                      8 days 03:07:00
Exposure Time [%]                    13.29037
Equity Final [$]                   9890.32224
Equity Peak [$]                   10000.39269
Commissions [$]                      82.99631
Return [%]                           -1.09678
Buy & Hold Return [%]                -3.94395
Return (Ann.) [%]                   -32.76802
Volatility (Ann.) [%]                 1.64567
CAGR [%]                            -28.95413
Sharpe Ratio                        -19.91165
Sortino Ratio                        -8.93002
Calmar Ratio                        -28.11572
Alpha [%]                            -1.00506
Beta                                  0.02326
Max. Drawdown [%]                    -1.16547
Avg. Drawdown [%]                    -0.58937
Max. Drawdown Duration        7 days 03:22:00
Avg. Drawdown Duration        3 days 13:43:00
# Trades

Backtest.run:   0%|          | 0/4402 [00:00<?, ?bar/s]


>>> REPORT: QQQ [5m] <<<
Start                     2025-09-26 17:10...
End                       2025-12-17 20:55...
Duration                     82 days 03:45:00
Exposure Time [%]                    37.29275
Equity Final [$]                   10209.1149
Equity Peak [$]                   10346.90877
Commissions [$]                     240.78616
Return [%]                            2.09115
Buy & Hold Return [%]                 0.79918
Return (Ann.) [%]                     9.34972
Volatility (Ann.) [%]                 4.06647
CAGR [%]                              6.55391
Sharpe Ratio                          2.29922
Sortino Ratio                         5.60189
Calmar Ratio                          7.02068
Alpha [%]                             2.01568
Beta                                  0.09443
Max. Drawdown [%]                    -1.33174
Avg. Drawdown [%]                    -0.12347
Max. Drawdown Duration       21 days 03:15:00
Avg. Drawdown Duration        1 days 07:45:00
# Trades

...


>>> REPORT: GC=F [1m] <<<
Start                     2025-12-11 08:20...
End                       2025-12-18 07:17...
Duration                      6 days 22:57:00
Exposure Time [%]                         0.0
Equity Final [$]                      10000.0
Equity Peak [$]                       10000.0
Return [%]                                0.0
Buy & Hold Return [%]                 2.78085
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
CAGR [%]                                  0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Alpha [%]                                 0.0
Beta                                      0.0
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rat

Backtest.run:   0%|          | 0/13088 [00:00<?, ?bar/s]


>>> REPORT: GC=F [5m] <<<
Start                     2025-10-09 20:45...
End                       2025-12-18 07:15...
Duration                     69 days 10:30:00
Exposure Time [%]                         0.0
Equity Final [$]                      10000.0
Equity Peak [$]                       10000.0
Return [%]                                0.0
Buy & Hold Return [%]                 9.27221
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
CAGR [%]                                  0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Alpha [%]                                 0.0
Beta                                      0.0
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rat

Backtest.run:   0%|          | 0/4264 [00:00<?, ?bar/s]


>>> REPORT: GC=F [15m] <<<
Start                     2025-10-13 07:45...
End                       2025-12-18 07:15...
Duration                     65 days 23:30:00
Exposure Time [%]                         0.0
Equity Final [$]                      10000.0
Equity Peak [$]                       10000.0
Return [%]                                0.0
Buy & Hold Return [%]                 6.64841
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
CAGR [%]                                  0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Alpha [%]                                 0.0
Beta                                      0.0
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Ra

In [62]:
# Membuat DataFrame Ranking
df_result = pd.DataFrame(summary_data)

# Urutkan berdasarkan Return Tertinggi (Bisa diubah ke Sharpe jika mau yang paling stabil)
df_ranked = df_result.sort_values(by='Return [%]', ascending=False).reset_index(drop=True)

print("\n===================================================================")
print("                 KESIMPULAN & ANALISIS PAKAR")
print("===================================================================")
print("\nTOP PERFORMA (Berdasarkan Return):")
display(df_ranked)

# Logika Rekomendasi
best_performer = df_ranked.iloc[0]
safest_performer = df_result.sort_values(by='Max Drawdown [%]', ascending=False).iloc[0] # Drawdown paling mendekati 0 (paling tidak minus)

print("\n--- ANALISIS METODE ---")
print(f"1. JUARA PROFIT: {best_performer['Ticker']} di Timeframe {best_performer['Timeframe']}")
print(f"   Alasan: Menghasilkan Return {best_performer['Return [%]']:.2f}% dengan Winrate {best_performer['Win Rate [%]']:.2f}%.")
print(f"   Karakteristik: Cocok untuk metode agresif ini karena volatilitasnya mendukung pyramiding.")

print(f"\n2. JUARA KEAMANAN (Safe Haven): {safest_performer['Ticker']} di Timeframe {safest_performer['Timeframe']}")
print(f"   Alasan: Max Drawdown hanya {safest_performer['Max Drawdown [%]']:.2f}%.")
print(f"   Catatan: Jika Anda ingin tidur nyenyak, gunakan aset dan timeframe ini.")

print("\n--- REKOMENDASI TIME FRAME ---")
avg_return_m1 = df_result[df_result['Timeframe'] == '1m']['Return [%]'].mean()
avg_return_m5 = df_result[df_result['Timeframe'] == '5m']['Return [%]'].mean()
avg_return_m15 = df_result[df_result['Timeframe'] == '15m']['Return [%]'].mean()

print(f"Rata-rata Return M1 : {avg_return_m1:.2f}% (Hanya 7 hari data)")
print(f"Rata-rata Return M5 : {avg_return_m5:.2f}% (60 hari data)")
print(f"Rata-rata Return M15: {avg_return_m15:.2f}% (60 hari data)")

if avg_return_m1 > avg_return_m5:
    print("-> Timeframe M1 terlihat lebih menguntungkan dalam jangka pendek, namun hati-hati noise.")
else:
    print("-> Timeframe M5/M15 lebih stabil dan konsisten untuk strategi trend following.")


                 KESIMPULAN & ANALISIS PAKAR

TOP PERFORMA (Berdasarkan Return):


,Ticker,Timeframe,Return [%],Win Rate [%],Sharpe Ratio,Max Drawdown [%],# Trades,Profit Factor,Duration
0,TSLA,5m,7.864769,45.517241,3.140205,-4.067048,145,1.695146,82 days 03:45:00
1,NVDA,5m,6.265993,33.913043,2.221944,-2.468553,115,1.899482,82 days 03:50:00
2,AMD,5m,5.271298,45.689655,1.810028,-3.512797,116,1.378156,82 days 03:40:00
3,NVDA,15m,3.518863,21.875000,1.420617,-4.466159,32,2.300703,75 days 03:00:00
4,TSLA,1m,3.035714,28.985507,5.165923,-1.961018,69,2.536122,8 days 03:05:00
5,AMZN,5m,2.842180,41.463415,2.215989,-1.671282,123,1.744749,82 days 03:45:00
6,AAPL,15m,2.429943,44.680851,1.558137,-3.185572,47,2.262334,75 days 03:00:00
7,GOOGL,15m,2.262463,35.849057,1.100090,-3.668948,53,1.350744,75 days 03:00:00
8,QQQ,5m,2.091149,44.262295,2.299225,-1.331739,122,1.792686,82 days 03:45:00
9,AMZN,15m,1.922100,39.215686,1.487135,-1.585625,51,1.406896,75 days 03:00:00



--- ANALISIS METODE ---
1. JUARA PROFIT: TSLA di Timeframe 5m
   Alasan: Menghasilkan Return 7.86% dengan Winrate 45.52%.
   Karakteristik: Cocok untuk metode agresif ini karena volatilitasnya mendukung pyramiding.

2. JUARA KEAMANAN (Safe Haven): GC=F di Timeframe 1m
   Alasan: Max Drawdown hanya -0.00%.
   Catatan: Jika Anda ingin tidur nyenyak, gunakan aset dan timeframe ini.

--- REKOMENDASI TIME FRAME ---
Rata-rata Return M1 : -0.81% (Hanya 7 hari data)
Rata-rata Return M5 : 2.26% (60 hari data)
Rata-rata Return M15: 1.17% (60 hari data)
-> Timeframe M5/M15 lebih stabil dan konsisten untuk strategi trend following.


In [63]:
# ==============================================================================
# SCRIPT VISUALISASI: HEAD-TO-HEAD COMPARISON (NVDA vs QQQ vs BTC)
# ==============================================================================
import plotly.graph_objects as go

# Kita pilih Timeframe M5 sebagai "Juara Umum"
target_tf = '5m'
assets_to_plot = ['NVDA', 'QQQ', 'BTC-USD']
equity_curves = {}

print(f"Sedang mengumpulkan data perbandingan timeframe {target_tf}...")

# Re-run simulasi singkat untuk mengambil data kurva ekuitas
for ticker in assets_to_plot:
    try:
        # 1. Ambil Data
        df = get_data(ticker, target_tf)
        if df is None: continue
        
        # 2. Setup Fitur & Macro
        df = add_technical_features(df)
        macro_df = train_macro_model(ticker)
        
        # Mapping Sinyal
        df['Macro_Signal'] = macro_df['Macro_Signal'].reindex(df.index, method='ffill')
        df.dropna(inplace=True)
        
        # 3. Jalankan Backtest Tanpa Print
        bt = Backtest(df, SmartPyramid, cash=10000, commission=0.0005, margin=1.0)
        stats = bt.run()
        
        # Simpan Kurva Ekuitas
        equity_curves[ticker] = stats._equity_curve['Equity']
        print(f"-> Data {ticker} diambil.")
        
    except Exception as e:
        print(f"Gagal memproses {ticker}: {e}")

# ================= BUAT GRAFIK INTERAKTIF =================
fig = go.Figure()

colors = {'NVDA': '#00FF00', 'QQQ': '#00BFFF', 'BTC-USD': '#FF4500'} # Hijau, Biru, Merah

for ticker, curve in equity_curves.items():
    # Normalisasi agar mulai dari titik 0% (biar adil perbandingannya)
    start_equity = curve.iloc[0]
    normalized_curve = (curve - start_equity) / start_equity * 100
    
    fig.add_trace(go.Scatter(
        x=normalized_curve.index, 
        y=normalized_curve,
        mode='lines',
        name=f'{ticker} ({target_tf})',
        line=dict(width=2, color=colors.get(ticker, 'white'))
    ))

fig.update_layout(
    title=f'Pertarungan Algoritma: Agresif (NVDA) vs Stabil (QQQ) vs Safety (BTC)',
    xaxis_title='Waktu',
    yaxis_title='Pertumbuhan Profit (%)',
    template='plotly_dark',
    hovermode='x unified',
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)
)

fig.show()

Sedang mengumpulkan data perbandingan timeframe 5m...
   [ML] Downloading data training untuk NVDA...


Backtest.run:   0%|          | 0/4433 [00:00<?, ?bar/s]

-> Data NVDA diambil.
   [ML] Downloading data training untuk QQQ...


Backtest.run:   0%|          | 0/4402 [00:00<?, ?bar/s]

-> Data QQQ diambil.
   [ML] Downloading data training untuk BTC-USD...


Backtest.run:   0%|          | 0/7842 [00:00<?, ?bar/s]

-> Data BTC-USD diambil.
